In [ ]:
%load_ext lab_black

In [ ]:
%matplotlib inline


from io import StringIO
from plotly.subplots import make_subplots
import bocd
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
import wget
import yfinance as yf
import zipfile

In [ ]:
BASE_URL = "https://data.binance.vision/data/futures/um/monthly/klines/{0}/{1}/"
COLUMNS = [
    "Open_time",
    "Open",
    "High",
    "Low",
    "Close",
    "Volume",
    "Close_time",
    "Quote_asset_volume",
    "Number_of_trades",
    "Taker_buy_base_asset_volume",
    "Taker_buy_quote_asset_volume",
    "Ignore",
]

### All intervals are supported: 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1mo

In [ ]:
def get_tickers(symbol: str, year: int, month: int, interval: str) -> pd.DataFrame:
    if not os.path.exists("./data"):
        os.mkdir("./data")

    filename = "{0}-{1}-{2}-{3}.zip".format(symbol, interval, year, str(month).zfill(2))
    filepath = "./data/" + filename
    if not os.path.exists(filepath):
        wget.download(
            BASE_URL.format(symbol, interval) + filename,
            out=filepath,
        )

    return pd.read_csv(filepath, header=None, names=COLUMNS)

In [ ]:
price_df = get_tickers(symbol="BTCBUSD", year=2021, month=4, interval="1h")

In [ ]:
signals = price_df["Close"].values
dates = price_df["Close_time"].values

In [ ]:
# Initialize object
bc = bocd.BayesianOnlineChangePointDetection(
    bocd.ConstantHazard(300), bocd.StudentT(mu=0, kappa=1, alpha=1, beta=1)
)

# Online estimation and get the maximum likelihood r_t at each time point
rt_mle = np.empty(signals.shape)
for i, d in enumerate(signals):
    bc.update(d)
    rt_mle[i] = bc.rt

# Project predictions on price
index_changes = np.where(np.diff(rt_mle) < 0)[0]
signal_changes = np.array(
    [None if i not in index_changes else v for i, v in enumerate(signals)]
)

In [ ]:
####################################
#        OHLC Plot with CPD        #
####################################
candle_fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    subplot_titles=("CPD", "OHLC"),
)

candle_fig.add_trace(
    go.Scatter(x=dates, y=signals, mode="lines", name="price"),
    row=1,
    col=1,
)
candle_fig.add_trace(
    go.Scatter(x=dates, y=signal_changes, mode="markers", name="cp"),
    row=1,
    col=1,
)

candle_fig.add_trace(
    go.Candlestick(
        x=dates,
        open=price_df["Open"],
        high=price_df["High"],
        low=price_df["Low"],
        close=price_df["Close"],
        name="OHLC",
    ),
    row=2,
    col=1,
)

candle_fig.update_layout(
    autosize=False,
    width=1400,
    height=700,
    margin=dict(l=10, r=10, b=20, t=40, pad=20),
    paper_bgcolor="LightSteelBlue",
)
candle_fig.show()